Utilizei Python 3 e as bibliotecas pandas, NumPy e Matplotlib.  Os códigos rodam no Jupyter Notebook. 

In [1]:
import pandas as pd

Aqui importo a biblioteca pandas e a nomeio como 'pd' para facilitar a digitação do código.

In [ ]:
xls = pd.ExcelFile('SIC 584922017103.xlsx')

Como é um arquivo Excel, uso a classe ExcelFile e chamo o dataframe de "xls".

In [ ]:
xls.sheet_names

Verifico as tabelas que existem dentro do arquivo. Existem 4 e as que me interessaram foi "Base de Dados" e "Campos da Tabela".

In [ ]:
xls.parse('Base de Dados')

Comando para checar o que compõe a tabela "Base de Dados". É realmente o que eu preciso, mas são muitas colunas (53), então é necessário uma limpeza.

In [ ]:
xls.parse('Campos da Tabela')

A tabela "Campos da Tabela" mostra o que significa cada coluna, então ficará mais fácil para entender cada uma e filtrar o que é realmente necessário para a história que quero contar.

In [ ]:
campos = pd.read_excel(xls, 'Campos da Tabela')

Criei um novo dataframe chamado "campos" somente com a tabela "Campos da Tabela", pois há muito espaço e dificultou a leitura de algumas linhas, então irei limpar.

In [ ]:
campos.columns

Confirmei o nome das colunas. Como não foi especificado na tabela, foi usado a primeira linha como nome de cada coluna. 

In [ ]:
campos = campos.rename(columns={'A PRESENTE TABELA TEM POR FINALIDADE ESCLARECER OS CAMPOS CONTIDOS NA BASE DE DADOS':'CAMPO'})
campos = campos.rename(columns={'Unnamed: 1':'DESCRIÇÃO'})
campos = campos.drop([0, 1, 2])

In [ ]:
campos

 Alterei o nome das colunas para algo mais amigável "CAMPO" e "DESCRIÇÃO" e retirei as três primeiras linhas da tabela.

In [ ]:
df = pd.read_excel(xls, 'Base de Dados')

Criei um novo dataframe chamado "df" somente com a tabela "Base de Dados".

In [ ]:
df.sample(5)

Mesmo com a descrição de cada campo, algumas coisas eu ainda não consegui entender, então peguei 5 exemplos aleatórios para analisar o que cada um significa e selecionar melhor as colunas que desejo manter.

In [ ]:
df = df[['CIDADE', 'DATA_COMUNICACAO_BO', 'FLAG_STATUS', 'VIOLENCIA_DOMESTICA', 'RUBRICA', 'DESCR_SUBTIPOLOCAL', 'DESCR_RELACIONAMENTO', 'IDADE_PESSOA', 'DESCR_PROFISSAO', 'DESCR_GRAU_INSTRUCAO']]

In [ ]:
df.sample(5)

Apenas com as colunas que desejo, fica mais fácil a análise dos dados. A forma como a idade está apresentada me incomoda um pouco, então vou corrigir.

In [ ]:
df.dtypes

O tipo da coluna está como float, mas vou mudar para inteiro, já que ninguem diz "minha idade é 51,3".

In [ ]:
import numpy as np

In [ ]:
df['IDADE_PESSOA'] = df['IDADE_PESSOA'].fillna(0).astype(np.int64)

Importei a biblioteca numpy como "np" e fiz a mudança do tipo da coluna, colocando "0" nas que não apresentavam valores.

In [ ]:
df.sample(5)

In [ ]:
df.to_csv('violencia-domestica-rp.csv', encoding='utf8', index=False)

É isso mesmo que quero, então aproveitei pra exportar a tabela caso eu queira utilizar novamente uma outra hora.

Como eu desejo falar especificamente do período da quarentena, preciso separar as datas por mês. A coluna "DATA_COMUNICACAO_BO" já está com o tipo correto, como vi anteriormente com o comando dtype, logo vai ser simples.

In [ ]:
df2019 = df[df['DATA_COMUNICACAO_BO'].dt.year == 2019]

In [ ]:
df2019.sample(5)

Primeiro, separei por ano. Irei fazer o mesmo com 2020. Perceba que criei um novo dataframe chamado "df2019" para não sobreescrever as informações do outro dataframe, criando também um novo dataframe para 2020, chamado "df2020"

In [ ]:
df2020 = df[df['DATA_COMUNICACAO_BO'].dt.year == 2020]

In [ ]:
df2020.sample(5)

In [ ]:
df2019 = df2019.sort_values(by='DATA_COMUNICACAO_BO')
df2020 = df2020.sort_values(by='DATA_COMUNICACAO_BO')

In [ ]:
df2019.tail(3)

In [ ]:
df2020.tail(3)

Primeiro, eu ordenei por data, do mês 1 ao 12. Depois, eu vi o final de cada uma para ver até qual mês iria. Como vai até o mês de julho na na tabela de 2020, farei o mesmo filtro na de 2019.

In [ ]:
dfquar19 = df2019[(df2019['DATA_COMUNICACAO_BO'] >= '2019-03-01') & (df2019['DATA_COMUNICACAO_BO'] <= '2019-06-30')]

In [ ]:
dfquar20 = df2020[(df2020['DATA_COMUNICACAO_BO'] >= '2020-03-01') & (df2020['DATA_COMUNICACAO_BO'] <= '2020-06-30')]

In [ ]:
dfquar19.head(5)

In [ ]:
dfquar20.sample(5)

Agora que tenho somente os meses que preciso, posso começar com minha matéria, fazendo a leitura dos dados nos dataframes específicos.

In [ ]:
dfquar19.to_csv('quar19.csv', index=False, encoding ='utf8')
dfquar20.to_csv('quar20.csv', index=False, encoding ='utf8')

In [ ]:
dfquar19.shape

In [ ]:
dfquar20.shape

Usei o comando shape para ver a quantidade de linhas em cada tabela, estimando, assim, o total de casos de cada ano.

In [ ]:
dfquar19['CIDADE'].unique()

Preciso filtrar por cidade, então tenho que ter certeza que os nomes estão escritos corretamente. Há um grande espaço neles, portanto irei remove-los.

In [ ]:
dfquar19['CIDADE'] = dfquar19['CIDADE'].map(str.strip)

In [ ]:
dfquar19['CIDADE'].unique()

Foi me dado um aviso, mas funcionou. Irei ignorar. Agora, filtrar.

In [ ]:
dfquar19rp = dfquar19[dfquar19['CIDADE']=='RIBEIRAO PRETO']

In [ ]:
dfquar19rp.sample(5)

Na coluna "CIDADE", peço apenas as linhas que contenham "RIBEIRAO PRETO". Agora tenho somente os crimes cometidos em Ribeirão Preto. Irei fazer mais algumas filtragens, com o mesmo comando usado anteriormente, e também no dataframe dfquar20. Achei melhor separar os dataframes, colocando um "rp" na frente.

In [ ]:
dfquar19rp.shape

In [ ]:
dfquar20['CIDADE'].unique()

In [ ]:
dfquar20['CIDADE'] = dfquar20['CIDADE'].map(str.strip)

In [ ]:
dfquar20['CIDADE'].unique()

In [ ]:
dfquar20rp = dfquar20[dfquar20['CIDADE']=='RIBEIRAO PRETO']

In [ ]:
dfquar20rp.sample(5)

In [ ]:
dfquar20rp.shape

In [ ]:
dfquar19rp['VIOLENCIA_DOMESTICA'].unique()

In [ ]:
dfquar19rpvd = dfquar19rp[dfquar19rp['VIOLENCIA_DOMESTICA']=='VIOLÊNCIA DOMESTICA']

In [ ]:
dfquar19rpvd.sample(5)

In [ ]:
dfquar19rpvd.shape

In [ ]:
dfquar20rp['VIOLENCIA_DOMESTICA'].unique()

In [ ]:
dfquar20rpvd = dfquar20rp[dfquar20rp['VIOLENCIA_DOMESTICA']=='VIOLÊNCIA DOMESTICA']

In [ ]:
dfquar20rpvd.sample(5)

In [ ]:
dfquar20rpvd.shape

Filtrei também pelos crimes que se enquadravam como violência doméstica e criei um novo dataframe para cada nomeados "dfquar19rpvd" e "dfquar20rpvd". Faço assim para não sobreescrever os outros dataframes, caso precise de alguma informação que esteja neles.

In [ ]:
dfquar19rpvd['DESCR_RELACIONAMENTO'].unique()

In [ ]:
dfquar19rpvd['DESCR_RELACIONAMENTO'] = dfquar19rpvd['DESCR_RELACIONAMENTO'].map(str.strip)

In [ ]:
dfquar19rpvd['DESCR_RELACIONAMENTO'].unique()

In [ ]:
dfquar19rpvd = dfquar19rpvd.set_index("DESCR_RELACIONAMENTO")

In [ ]:
dfquar19rpvd = dfquar19rpvd.drop("CONHECIDO", axis=0)
dfquar19rpvd = dfquar19rpvd.drop("PARENTESCO", axis=0)
dfquar19rpvd = dfquar19rpvd.drop("VIZINHANCA", axis=0)
dfquar19rpvd = dfquar19rpvd.drop("NENHUMA RELACAO", axis=0)
dfquar19rpvd = dfquar19rpvd.drop("IGNORADO", axis=0)

In [ ]:
dfquar19rpvd.sample(5)

In [ ]:
dfquar19rpvd.shape

Com estes comandos, eu transformei a coluna "DESCR_RELACIONAMENTO" em índice para facilitar a filtragem por linhas, selecionando apenas os registros onde a vítima tinha algum relacionamento amoroso com o agressor. Farei o mesmo com o dataframe "dfquar20rpvd"

In [ ]:
dfquar20rpvd['DESCR_RELACIONAMENTO'].unique()

In [ ]:
dfquar20rpvd = dfquar20rpvd.dropna(subset=['DESCR_RELACIONAMENTO'])

In [ ]:
dfquar20rpvd['DESCR_RELACIONAMENTO'] = dfquar20rpvd['DESCR_RELACIONAMENTO'].map(str.strip)

Por conter um valor NaN na coluna "DESCR_RELACIONAMENTO", eu não conseguiria remover os espaços. Então eu excluí as linhas com esse valor e depois os removi.

In [ ]:
dfquar20rpvd = dfquar20rpvd.set_index("DESCR_RELACIONAMENTO")

In [ ]:
dfquar20rpvd = dfquar20rpvd.drop("CONHECIDO", axis=0)
dfquar20rpvd = dfquar20rpvd.drop("PARENTESCO", axis=0)
dfquar20rpvd = dfquar20rpvd.drop("NENHUMA RELACAO", axis=0)
dfquar20rpvd = dfquar20rpvd.drop("IGNORADO", axis=0)

In [ ]:
dfquar20rpvd.shape

In [ ]:
dfquar19rpvd = dfquar19rpvd.reset_index()
dfquar20rpvd = dfquar20rpvd.reset_index()

Voltei para o índice padrão nos dois dataframes.

In [ ]:
import matplotlib.pyplot as plt

Importei a biblioteca Matplotlib e a chamei de plt. Ela serve para gerar visualizações.

In [ ]:
dfquar19rpvd['IDADE_PESSOA'].value_counts()

Qual a idade mais frequente na coluna "IDADE_PESSOA".

In [ ]:
dfquar19rpvd['IDADE_PESSOA'].value_counts()[:10].plot.barh(color='#4874C2')

Gerei um pequeno gráfico com apenas os 10 primeiros valores mais frequentes.

In [ ]:
dfquar20rpvd['IDADE_PESSOA'].value_counts()

In [ ]:
dfquar20rpvd['IDADE_PESSOA'].value_counts()[:10].plot.barh(color='#45A2D9')

O mesmo para o dataframe "dfquar20rpdv".

In [ ]:
dfquar19rpvd['RUBRICA'].value_counts()

In [ ]:
dfquar20rpvd['RUBRICA'].value_counts()

O mesmo para "RUBRICA".

In [ ]:
dfquar19rp['RUBRICA'].value_counts()

In [ ]:
dfquar20rp['RUBRICA'].value_counts()

O mesmo no dataframe maior, para comparação.

In [ ]:
dfquar19['DESCR_GRAU_INSTRUCAO'].value_counts()

In [ ]:
dfquar20['DESCR_GRAU_INSTRUCAO'].value_counts()

O grau de instrução.

Com os dados em mãos, vou gerar visualizações. Criarei um único dataframe com eles para facilitar. 

In [ ]:
dados = [['Região 2019', 13774], ['Região 2020', 9876], ['Ribeirão Preto 2019', 2320], ['Ribeirão Preto 2020', 1551]] 

In [ ]:
total = pd.DataFrame(dados, columns = ['Descrição', 'Quantidade']) 

In [ ]:
total

In [ ]:
total = total.set_index("Descrição")

In [ ]:
total.plot.barh(grid=True, color='#5942CF')

Parece complicado e lento, mas é muito mais prático utilizar apenas o teclado para criar uma tabala e não ter que ficar pulando células. Bom, eu criei uma variável chamada "dados" e dei uma descrição e o valor. Abaixo, criei duas colunas, "Descrição" e "Quantidade". Depois, transforei a "Descrição" em índice e, por fim, gerei a visualização. 

In [ ]:
viodom = [['2019', 1112], ['2020', 818]]

In [ ]:
dfviodom = pd.DataFrame(viodom, columns = ['Ano', 'Quantidade']) 

In [ ]:
dfviodom

In [ ]:
dfviodom = dfviodom.set_index("Ano")

In [ ]:
dfviodom.plot.bar(grid=True, color='#4555D9')

Aí o trabalho vai se repetindo e você pode montar o que desejar com os dados.